# Understanding the data
There is a lot of information in these files, most of which I don't understand.  Can we figure out which rows are associated with each proposal?  Can we determine the status of the proposal?

This workspace also includes progress towards transferring the data to a relational database.  But all that code should also be in database_creator.py

-Tristan

# Setup

In [1]:
# packages
import datetime
import matplotlib as mp
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import re
import sys

# seaborn settings
sns.set()
sns.set_style('whitegrid')
sns.set_style('ticks')

In [2]:
# load data
fields = pd.read_csv('planning-department-records-2018/DataSF_PPTS_Fields.csv')
data = pd.read_csv('planning-department-records-2018/PPTS_Records_data.csv',
#                   parse_dates=['date_opened','date_closed',],
                   infer_datetime_format=True)

C:\Users\Tristan\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2723: DtypeWarning: Columns (28,31,32,33,34,36,38,39,40,41,42,43,44,45,46,48,119,163,167,168,169) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
datacopy = data.copy()

In [5]:
if 'database_creator' in sys.modules:
    del sys.modules['database_creator']
from database_creator import *

# Record types
Through some sleuthing, I've determined that there is a hierarchy of categorization.

- record_type_group: 'planning' for all data
- record_type_type: 5 categories: 'Project', 'Applications', 'Complaint', 'Other', 'Research'
- record_type_subtype: 17 categories
- record_type: 63 categories.  Each one, except for 'Other' is given a 3-letter abbreviation.  The abbreviations are in the column record_type_category.  Note that there are a bunch of typos in these two columns, mostly when people fail to follow a consistent format.
- record_type_4level contains the entire hierarchy

In [82]:
sorted(data['record_type_4level'].unique())

['Planning/Applications/Appeal/APL',
 'Planning/Applications/Citywide Planning/CWP',
 'Planning/Applications/Citywide Planning/GPL',
 'Planning/Applications/Commission Review/CRV',
 'Planning/Applications/Commission Review/IMP',
 'Planning/Applications/Discretionary Review/DRM',
 'Planning/Applications/Discretionary Review/DRP',
 'Planning/Applications/Environmental/CATEX-EEC',
 'Planning/Applications/Environmental/Community Plan Determin-ECD',
 'Planning/Applications/Environmental/Community Plan Exemption-ECE',
 'Planning/Applications/Environmental/Community Plan Focused EIR-EIF',
 'Planning/Applications/Environmental/EIR Addendum-EIA',
 'Planning/Applications/Environmental/EIR Fee',
 'Planning/Applications/Environmental/ENV',
 'Planning/Applications/Environmental/Environmental T and M-ETM',
 'Planning/Applications/Environmental/Initial Study-EEA',
 'Planning/Applications/Environmental/Neg Dec Addendum-ENA',
 'Planning/Applications/Environmental/Transportation Abbrev-ETA',
 'Planning/

In [8]:
# make a dict expanding out each abbreviation.
# also, print out any anomalies in the record_type column.

# note that there are also anomalies in the record_type_category column
# it seems it's supposed to contain 3-letter abbreviations, but the environmental department doesn't follow that format.
record_type_dict = {}

oddity_list = []
record_abbr = {}
for record_type in data['record_type'].unique():
    m = re.match('^(.*) \((...)\)',record_type)
    if(m):
        if m.groups()[1] in record_abbr:
            oddity_list.append(record_type)
        else:
            record_abbr[m.groups()[1]] = m.groups()[0]
    else:
        oddity_list.append(record_type)

record_type_dict['Other'] = 'Other'

print(oddity_list)

['Environmental', 'Conditional Use Authorization', 'Variance', 'General Plan Referral', 'Discretionary Review - Public Initiated', 'Other', 'Discretionary Review - Manditory (DRM)']


# Standardizing record_type
With an eye towards creating a relational database, I'm writing code to create a record_type table and standardize the record_type_category column

In [31]:
#create a dataframe for record types
record_type = record_type_table(data)

In [36]:
#original_type contains original record_type_category
#record_type contains original_type after format standardization
record_type.head()

,original_type,record_type_name,record_type_cat,record_type_subcat,record_type
0,PRJ,Project Profile (PRJ),Project,Project,PRJ
1,ENV,Environmental (ENV),Applications,Environmental,ENV
2,MIS,Misc. Permits-REF (MIS),Applications,Referral,MIS
3,COA,Certificate of Appropriateness (COA),Applications,Planning Entitlements,COA
4,ENF,Enforcement (ENF),Complaint,Code Enforcement,ENF


In [53]:
#use record_type table to rename the record_type_category column
data = clean_record_type(data,record_type)

# Families of associated entries
Each proposal should be associated with several entries.  For instance, there may be one for the project proposal, one for the research, several for applications, etc.  Here I try to group entries into families.

The first method I tried is to look at parents or children.  It seems that this is not always reliable.  Another method is to look at addresses, but this is also not reliable because sometimes the address is nan, or it appears in different formats.  A more reliable method is to use the_geom, which is never nan.  However, the_geom picks out buildings, not projects.  A single building may have a lot of unrelated paperwork associated with it.

In [57]:
#return a list of rows of family members, based on parent/children columns
def get_family(data,row):
    members = {}
    find_family(data,members,row,1)
    return list(members.values())

def find_family(data,members,curr_row,stacklevel):
    if stacklevel>40:
        print('Error: too many recursions on row %i' % curr_row)
        return
    members[data.loc[curr_row,'record_id']] = curr_row
    family = []
    if not pd.isna(data.loc[curr_row,'parent']):
        family += data.loc[curr_row,'parent'].split(',')
    if not pd.isna(data.loc[curr_row,'children']):
        family += data.loc[curr_row,'children'].split(',')
    for member in family:
        if member not in members:
            #find row of the member
            member_match = np.where(data['record_id'] == member)[0]
            if(len(member_match > 0)):
                find_family(data,members,member_match[0],stacklevel+1)


In [286]:
#return a list of rows associated with the same building
def get_building(data,row):
    my_geom = data.loc[row,'the_geom']
    member_match = np.where(data['the_geom'] == my_geom)[0]
    return list(member_match)

#return a list of rows associated with the same address
#unfortunately, addresses aren't always in a consistent format
def get_address(data,row):
    my_address = data.loc[row,'address']
    member_match = np.where(data['address'] == my_address)[0]
    return list(member_match)

In [287]:
#takes the output of get_building, and divides the list into families
#returns a dict with each index as a key, and the family number as the value
def divide_by_family(data,building):
    if type(building) != list:
        building = get_building(data,building)
    family_ids = {}
    i = 1
    for b in building:
        if b not in family_ids:
            family = get_family(data,b)
            for member in family:
                family_ids[member] = i
            i += 1
    return(family_ids)

# Location and record_rel tables
For the relational database, need to create tables for locations, and relations between records.

In [97]:
record_rel = record_rel_table(data)

In [98]:
record_rel.head()

,child_id,parent_id
0,0,132846
1,0,110576
2,0,164661
3,6,152516
4,18,39078


In [103]:
record_rel.shape

(47917, 2)

In [100]:
#make sure it works by testing consistency with get_family
get_family(data,0)

[164661, 0, 132846, 110576]

In [141]:
if 'database_creator' in sys.modules:
    del sys.modules['database_creator']
from database_creator import *

In [144]:
#generate location table
comp_timer = timer()
data, location = location_table(data)
comp_timer.pause()

In [152]:
print('%.2f min' % comp_timer.report())

7.07 min


In [146]:
data.loc[10000:10005,'location_id']

10000    8300
10001    8301
10002    3544
10003    8302
10004    3119
10005    8303
Name: location_id, dtype: int64

In [147]:
location.head()

,the_geom,location_id,shape_length,shape_area,address
0,"MULTIPOLYGON (((-122.438540755 37.744993973, -...",20690.0,0.000837,3.404328e-08,825 DUNCAN ST 94131
1,"MULTIPOLYGON (((-122.418368736 37.797849736, -...",54655.0,0.000907,2.675426e-08,"1175 GREEN ST, SAN FRANCISCO, CA 94109"
2,"MULTIPOLYGON (((-122.427835182 37.754438944, -...",13236.0,0.000829,2.374917e-08,"1020 CHURCH ST, #0001, SAN FRANCISCO, CA 94114"
3,"MULTIPOLYGON (((-122.383204945 37.730905356, -...",54227.0,0.032722,4.102967e-06,"0 INGALLS ST, SAN FRANCISCO, CA"
4,"MULTIPOLYGON (((-122.433177812 37.752553624, -...",2777.0,0.000799,2.703637e-08,"4055 23RD ST, SAN FRANCISCO, CA 94114"


# Building stories
I thought I'd look at a few examples of buildings, to get a sense of what they look like.

Overall, my impression is that there are a lot of unrelated stories going on with each building.  Sometimes there are multiple project proposals, sometimes there are a bunch of miscellaneous stuff like complaints.  I'm still not sure how to determine the status of a project, but if the information is spread across multiple rows it may be difficult to figure out which rows to look at.  No, the 'family' doesn't help, I checked.

In [126]:
def tell_story(data,row):
    building = get_building(data,row)
    location = data.loc[building,:]
    family_ids = divide_by_family(data,building)
    #print the first valid address
    for i in range(location.shape[0]):
        if not pd.isna(location['address'].iloc[i]):
            print(location['address'].iloc[i])
            break
    location.sort_values('date_opened',inplace=True)
    for i in range(location.shape[0]):
        print('')
        #print('family %s' % family_ids[location.index[i]],location['date_opened'].iloc[i], location['record_type'].iloc[i], location['record_status'].iloc[i])
        print(location['date_opened'].iloc[i], location['record_type'].iloc[i], location['record_status'].iloc[i])
        print(location['description'].iloc[i])

In [68]:
i = 82
print(get_building(data,i))

[82, 5861, 5864, 9766, 18982, 28271, 49426, 89982, 98653, 108095, 138800, 145264, 150667, 152836, 163170, 169650, 178576, 183313]


In [6]:
#we can see that even though it's just one building, the address is inconsistent.
data.loc[get_building(data,i),'address']

82                                        234 01ST ST 94105
5861                                      246 01ST ST 94105
5864                                      234 01ST ST 94105
9766                                      234 01ST ST 94105
18982                  234 01ST ST, SAN FRANCISCO, CA 94105
28271      246 01ST ST, #3RD FLOOR, SAN FRANCISCO, CA 94105
49426                  234 01ST ST, SAN FRANCISCO, CA 94105
89982                  234 01ST ST, SAN FRANCISCO, CA 94105
98653                                     234 01ST ST 94105
108095                                    246 01ST ST 94105
138800                                    246 01ST ST 94105
145264                                    246 01ST ST 94105
150667                                    246 01ST ST 94105
152836                                    246 01ST ST 94105
163170    246 01ST ST, #STREET LEV, SAN FRANCISCO, CA 94105
169650                                    246 01ST ST 94105
178576                                  

In [127]:
#I'm commenting out the part where I print the story, to reduce clutter
#tell_story(data,82)

234 1st St is an apartment building (which also has the address 246 1st St).  In 2006 there was a permit to create a massage establishment on the ground floor.  In 2008, there was a permit to move a tenant on the third floor.  There are a bunch of permits and project proposals related to AT&T and T-mobile wanting to replace equipment at the top of the building.  In 2015, there was a proposal to renovate the building.  It starts out as a PRV (project review meeting), followed by a PRJ (project profile).  A year later they file a PTA (permit to alter), which is approved.  However, the status of the project is marked as "under review" which might mean they still haven't gotten permission.

In 2017, there are a few extra entries.  There are two letters: one TDU (notice of use), and one ZAV (zoning verification).  And it seems like someone applied for it to be designated as a historical landmark, which was approved.

TL;DR: there are a lot of things going on in a single building.  There was one major construction proposal, which I don't think ever got through.  And I wonder if opponents of the proposal made the building into a landmark.

In [70]:
#tell_story(data,3)

900 Sansome is a public parking lot.  In 2009 there was one project to replace a window, which had to get through COA and CUA, but is apparently closed, so maybe it passed?  Most of the rest seems to be an annual renewal of some permit for parking facilities.  There are also a few complaints about signs being wrong

In [71]:
#tell_story(data,0)

875 Sansome is currently a two-story office space.  Since 2016 they have been looking into replacing it with a 6-story mixed use building.  A project profile was finally made in 2018, and it would add 6 residential units (which I take it means that all of them are market rate).  The CUA is currently under review.

In [95]:
#tell_story(data,23499)

383 King St is a condo building.  The project to construct it was started in 2006, and I presume that it finished.  Later, in 2016 and later, there are several complaints about illegal short-term rentals through AirBnB.

# Project statuses
Here I try to determine what kinds of statuses a project can have.  Work in progress.  I think I'll have to e-mail the planning department.

In [98]:
data.loc[data['record_type_category']=='PRJ','record_status'].unique()

array(['Accepted', 'Withdrawn', 'Closed', 'Closed - Withdrawn', 'Open',
       'Under Review', 'Application Accepted', nan, 'Complete',
       'Submitted', 'Pending', 'Closed - Approved', 'Approved', 'On Hold',
       'Incomplete', 'Actions Pending', 'Pending Review', 'Denied',
       'Disapproved', 'Closed - Complete'], dtype=object)

# Planner table
Another table for the database

In [91]:
#one tricky part is that the id, e-mail, name, and phone aren't always given
print(sum(~pd.isna(data['planner_id'])))
print(sum(~pd.isna(data['planner_email'])))
print(sum(~pd.isna(data['planner_name'])))
print(sum(~pd.isna(data['planner_phone'])))
#after some testing, it seems that this is consistent for any given planner_id.
#I guess a lot of planners just have some info unavailable.

140186
83021
84843
84630


In [73]:
if 'database_creator' in sys.modules:
    del sys.modules['database_creator']
from database_creator import *

In [200]:
comp_timer = timer()
data, planner = planner_table(data)
comp_timer.pause()
comp_timer.printreport()

Time passed: 0.10 min


In [106]:
planner.head()

,planner_strid,planner_id,planner_email,planner_phone,planner_name
0,BS,385.0,NaN,NaN,NaN
1,GWONG,171.0,NaN,NaN,NaN
2,HGITELMA,196.0,NaN,NaN,NaN
3,DSIDER,197.0,dan.sider@sfgov.org,415-558-6697,Dan Sider
4,JSHAMBRA,3.0,NaN,NaN,NaN


# Tables for sparse features
Something that really blows up the size of the data, is that there are a bunch of columns which are very sparsely populated.  We have:

- prj_desc: Project Descriptions.  Mostly a bunch of checkboxes in these forms that people fill out.  There are two exceptions, namely MCD_REFERRAL and ENVIRONMENTAL_REVIEW_TYPE, which have written fields instead of checkboxes.  The written fields will be given an additional table
- land_use: For PRJ records, a measure of pre-existing, proposed, and net change in land use, in many land use categories.  Always in square feet.
- prj_feature: For PRJ records, a measure of pre-existing, proposed, and net change number of units of various types.  e.g. parking spaces or affordable housing units.  There's also a space for people to write in some "other" feature.  And there are a couple "proposed" columns which are empty for some reason.
- dwelling: For PRJ records, a measure of pre-existing, proposed, and net change number of residential dwelling units of various types.  e.g. 1BR, 2BR.  ADUs (Accessory Dwelling Units) also list an area which I will put in the adu_area table.

In [122]:
#first, let's fix those two broken prj_feature columns
data = fix_prj_features(data)

In [236]:
#create list of column names for hardcoding purposes
prj_desc_str = ''
for i in range(27,51):
    prj_desc_str += '"' + fields.loc[i,'Field'] + '", '
print(prj_desc_str)

"CHANGE_OF_USE", "ADDITIONS", "NEW_CONSTRUCTION", "LEG_ZONE_CHANGE", "DEMOLITION", "LOT_LINE_ADJUST", "FACADE_ALT", "ROW_IMPROVE", "OTHER_PRJ_DESC", "SPECIAL_NEEDS", "SENIOR", "AFFORDABLE_UNITS", "STUDENT", "INCLUSIONARY", "STATE_DENSITY_BONUS", "ADU", "FORMULA_RETAIL", "MCD", "TOBACCO", "FINANCIAL", "MASSAGE", "MCD_REFERRAL", "OTHER_NON_RES", "ENVIRONMENTAL_REVIEW_TYPE", 


In [74]:
prj_desc, prj_desc_detail = prj_desc_table(data)

C:\Users\Tristan\Anaconda3\lib\site-packages\pandas\core\ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


In [173]:
#here, prj_desc_id refers to the index of prj_desc
prj_desc_detail.head()

,detail,prj_desc_id
0,Restaurant,13024
1,Medical Cannabis Dispensary,13025
2,Restaurant,13026
3,Limited-Restaurant,13027
4,Bar,13028


In [182]:
#here, record_id refers to the index of data
prj_desc.head()

,desc_type,record_id
0,CHANGE_OF_USE,0
1,CHANGE_OF_USE,133
2,CHANGE_OF_USE,152
3,CHANGE_OF_USE,166
4,CHANGE_OF_USE,312


In [266]:
#next let's do land_use
land_use_str = ''
for i in range(51,75,3):
    land_use_str += '"' + fields.loc[i,'Field'][9:-6] + '", '
print(land_use_str)

"RC", "RESIDENTIAL", "CIE", "PDR", "OFFICE", "MEDICAL", "VISITOR", "PARKING_SPACES", 


In [281]:
if 'database_creator' in sys.modules:
    del sys.modules['database_creator']
from database_creator import *

In [233]:
land_use = land_use_table(data)

In [234]:
land_use.head()

,land_use_exist,land_use_net,land_use_prop,land_use_type,record_id
0,0.0,2840.0,2840.0,LAND_USE_RC,0
1,0.0,1740.0,1740.0,LAND_USE_RC,133
2,2747.0,-1587.0,1160.0,LAND_USE_RC,173
3,940.0,0.0,940.0,LAND_USE_RC,225
4,20417.0,1868.0,22285.0,LAND_USE_RC,240


In [265]:
#next project features
prj_feature_str = ''
for i in range(75,120,3):
    prj_feature_str += '"' + fields.loc[i,'Field'][12:-6] + '", '
print(prj_feature_str)
#note: there's a mistake in the fields table, which results in "STORIES" becoming "STORIE"

"AFFORDABLE", "HOTEL_ROOMS", "MARKET_RATE", "BUILD", "STORIE", "PARKING", "LOADING", "BIKE", "CAR_SHARE", "USABLE", "PUBLIC", "ART", "ROOF", "SOLAR", "LIVING", 


In [270]:
prj_feature = prj_feature_table(data)

In [271]:
prj_feature.head()

,prj_feature_exist,prj_feature_net,prj_feature_prop,prj_feature_type,record_id
0,0.0,6.0,6.0,AFFORDABLE,133
1,8.0,2.0,10.0,AFFORDABLE,233
2,1.0,0.0,1.0,AFFORDABLE,273
3,198.0,0.0,198.0,AFFORDABLE,469
4,13.0,0.0,13.0,AFFORDABLE,688


In [276]:
#finally, dwelling types
prj_feature_str = ''
for i in range(124,136,3):
    prj_feature_str += '"' + fields.loc[i,'Field'][12:-6] + '", '
for i in range(136,152,4):
    prj_feature_str += '"' + fields.loc[i,'Field'][12:-6] + '", '
for i in range(152,164,3):
    prj_feature_str += '"' + fields.loc[i,'Field'][12:-6] + '", '
print(prj_feature_str)

"STUDIO", "1BR", "2BR", "3BR", "ADU_STUDIO", "ADU_1BR", "ADU_2BR", "ADU_3BR", "GH_ROOMS", "GH_BEDS", "SRO", "MICRO", 


In [282]:
dwelling, adu_area = dwelling_table(data)

In [283]:
dwelling.head()

,dwelling_exist,dwelling_net,dwelling_prop,dwelling_type,record_id
0,0.0,1.0,1.0,STUDIO,152
1,12.0,2.0,14.0,STUDIO,276
2,19.0,0.0,19.0,STUDIO,469
3,0.0,35.0,35.0,STUDIO,2422
4,45.0,0.0,45.0,STUDIO,3085


In [284]:
adu_area.head()

,adu_area,dwelling_id
0,707.0,2085
1,242.0,2086
2,389.0,2087
3,360.0,2088
4,308.0,2089


# Calculating year/month/day

In [43]:
if 'database_creator' in sys.modules:
    del sys.modules['database_creator']
from database_creator import *

In [109]:
data = ymd(data)

# Transferring to a SQL database

In [11]:
data, record_type, record_rel, location, planner, prj_desc, prj_desc_detail, land_use, prj_feature, dwelling, adu_area = prepare_data(data)

Generating record_type table
Time passed: 0.12 min
Generating record_rel table
Time passed: 0.02 min
Generating location table
Time passed: 8.05 min
Generating planner table
Time passed: 0.11 min
Generating prj_desc tables


C:\Users\Tristan\Anaconda3\lib\site-packages\pandas\core\ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


Time passed: 0.01 min
Generating land_use table
Time passed: 0.01 min
Generating prj_feature table
Time passed: 0.24 min
Generating dwelling tables
Time passed: 0.02 min
Generating year/month/day
Time passed: 0.45 min


In [48]:
init_sql_database('test9.db', data, record_type, record_rel, location, planner, prj_desc,
                      prj_desc_detail, land_use, prj_feature, dwelling, adu_area)

In [55]:
#just some code for testing
import sqlite3 as lite
con = lite.connect('test9.db')
with con:
    cur = con.cursor()
    cur.execute('select area from adu_area where dwelling_id < 2100')
    print(cur.fetchall())

[(707.0,), (242.0,), (389.0,), (360.0,), (308.0,), (None,), (352.0,), (433.0,), (459.0,), (229.0,), (None,), (330.0,), (420.0,), (None,), (624.0,)]


In [56]:
adu_area.loc[0:10,'area']

0     707.0
1     242.0
2     389.0
3     360.0
4     308.0
5       NaN
6     352.0
7     433.0
8     459.0
9     229.0
10      NaN
Name: area, dtype: float64